## Install all required packages

In [4]:
!pip install chromadb sentence-transformers azure-ai-inference openpyxl pandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currentl

## Add your GITHUB PAT TOKEN to the secrets in google colab and then run this cell

In [6]:
from google.colab import userdata
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

## Import all the necessary libraries and read the Excel file.

In [7]:
import os
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# path to your uploaded excel file (upload via the Colab file panel on the left)
EXCEL_FILE_PATH = "universities.xlsx"  # change filename if yours is different

In [8]:
# load the excel file
df = pd.read_excel(EXCEL_FILE_PATH)

# fill blank/NaN values so they dont cause issues
df = df.fillna("Not specified")

# print shape and columns so you can verify it loaded correctly
print(f"Loaded {len(df)} rows and {len(df.columns)} columns")
print("Columns:", list(df.columns))
print("\nSample row:\n", df.iloc[0])

Loaded 30 rows and 21 columns
Columns: ['         University Name', '          City', '               Course', '     Deadline', '     Ilets', 'GPA german', ' credits', 'website link daad', '    vpd', 'Application fee', ' Specific requirement', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']

Sample row:
          University Name                                  University of trier
          City                                                          trier
               Course                             Natural language processing
     Deadline                                           6th May - 31 may 2025
     Ilets                                                                6.5
GPA german                                                      Not specified
 credits                                                        Not specified
website link daad           https://www2.daad.

## To create the Vector DB convert each row to readable text and load the embedding model to generate embeddings for all rows.

In [9]:
def row_to_text(row):
    # converts a single dataframe row into a readable text string
    # this becomes one "chunk" in the vector DB
    parts = []
    for col, val in row.items():
        parts.append(f"{col}: {val}")
    return " | ".join(parts)

# convert every row into a text chunk
chunks = []
chunk_ids = []

for i, row in df.iterrows():
    text = row_to_text(row)
    chunks.append(text)
    chunk_ids.append(str(i))

print(f"\nTotal chunks created: {len(chunks)}")
print("\nExample chunk:\n", chunks[0])


Total chunks created: 30

Example chunk:
          University Name: University of trier |           City: trier |                Course: Natural language processing |      Deadline: 6th May - 31 may 2025 |      Ilets: 6.5 | GPA german: Not specified |  credits: Not specified | website link daad: https://www2.daad.de/deutschland/studienangebote/international-programmes/en/detail/7708/ |     vpd: not required | Application fee: NO (uni-portal) |  Specific requirement: a suitable Bachelor's degree (acquisition of at least 180 ECTS from a domestic or foreign university) | Unnamed: 11: Not specified | Unnamed: 12: Not specified | Unnamed: 13: Not specified | Unnamed: 14: Not specified | Unnamed: 15: Not specified | Unnamed: 16: Not specified | Unnamed: 17: Not specified | Unnamed: 18: Not specified | Unnamed: 19: Not specified | Unnamed: 20: Not specified


In [10]:
# loading a lightweight local embedding model
# this runs entirely on the colab machine, no API key needed
print("Loading embedding model...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# create embeddings for all chunks
print("Generating embeddings for all university rows...")
embeddings = embedder.encode(chunks, show_progress_bar=True)

# set up chromadb in-memory
chroma_client = chromadb.Client()

# create a collection (like a table in a DB)
collection = chroma_client.create_collection(name="universities")

# add all chunks with their embeddings into the collection
collection.add(
    documents=chunks,
    embeddings=embeddings.tolist(),
    ids=chunk_ids
)

print(f"\nVector DB ready. Total documents indexed: {collection.count()}")

Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings for all university rows...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Vector DB ready. Total documents indexed: 30


## Set up the github marketplace LLM client

In [25]:
llm_client = ChatCompletionsClient(
    endpoint="https://models.github.ai/inference",
    credential=AzureKeyCredential(GITHUB_TOKEN),
)

## Keyword search scans every chunk for exact word matches from the query and ranks them by match count.

In [26]:
import re

def keyword_search(query, top_k=4):
    # simple keyword search directly on the original dataframe
    # splits query into individual words and checks if any row contains them
    query_words = [w.lower().strip() for w in query.split() if len(w) > 2]

    scores = []
    for i, chunk in enumerate(chunks):
        chunk_lower = chunk.lower()
        # count how many query words appear in this chunk
        match_count = sum(1 for word in query_words if word in chunk_lower)
        if match_count > 0:
            scores.append((i, match_count))

    # sort by match count descending
    scores.sort(key=lambda x: x[1], reverse=True)

    top_indices = [str(idx) for idx, _ in scores[:top_k]]
    top_chunks = [chunks[idx] for idx, _ in scores[:top_k]]
    return top_indices, top_chunks

## Hybrid retrieval combines semantic search from ChromaDB and keyword search, merges both results and deduplicates them.

In [27]:
def retrieve_hybrid(query, top_k=4, verbose=False):
    # step 1: semantic search via chromadb
    query_embedding = embedder.encode([query]).tolist()
    semantic_results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k
    )
    semantic_ids = semantic_results["ids"][0]
    semantic_chunks = semantic_results["documents"][0]

    # step 2: keyword search on raw chunks
    keyword_ids, keyword_chunks = keyword_search(query, top_k=top_k)

    # step 3: merge both result sets, deduplicate, preserve order
    # keyword results go first so they dont get buried if semantics failed
    seen_ids = set()
    merged_chunks = []

    for id_, chunk in zip(keyword_ids, keyword_chunks):
        if id_ not in seen_ids:
            seen_ids.add(id_)
            merged_chunks.append(("keyword", chunk))

    for id_, chunk in zip(semantic_ids, semantic_chunks):
        if id_ not in seen_ids:
            seen_ids.add(id_)
            merged_chunks.append(("semantic", chunk))

    if verbose:
        print("--- HYBRID RETRIEVAL RESULTS ---")
        for source, chunk in merged_chunks:
            print(f"[{source}] {chunk}\n")
        print("--------------------------------\n")

    # return just the chunks for the LLM
    return [chunk for _, chunk in merged_chunks]

## ask_rag ties everything together, builds the prompt with retrieved context and calls the LLM to get the final answer.

In [28]:
def ask_rag(question, top_k=4, verbose=False):
    # uses hybrid retrieval instead of pure semantic
    relevant_chunks = retrieve_hybrid(question, top_k=top_k, verbose=verbose)

    context = "\n\n".join([f"Entry {i+1}: {chunk}" for i, chunk in enumerate(relevant_chunks)])

    prompt = f"""You are a helpful assistant that answers questions about German university admissions.
Use ONLY the information provided below to answer the question. If the answer is not in the context, say "I don't have that information in the dataset."

Context from the university dataset:
{context}

Question: {question}
Answer:"""

    response = llm_client.complete(
        messages=[
            SystemMessage("You are a helpful assistant for German university admissions. Answer based only on the provided context."),
            UserMessage(prompt),
        ],
        model="meta/Llama-3.3-70B-Instruct",
        temperature=0.3,
        max_tokens=600,
        top_p=0.9
    )

    return response.choices[0].message.content

## Alas Testing the RAG with question.

In [24]:
# change the question here and just re-run this single cell
# set verbose=True if you want to see which university rows were retrieved
question = "which universities accept IELTS 6.5 for artificial intelligence and have deadlines after April"
answer = ask_rag(question, top_k=4)

print("Question:", question)
print("\nAnswer:", answer)

Question: which universities accept IELTS 6.5 for artificial intelligence and have deadlines after April

Answer: Based on the provided context, the universities that accept IELTS 6.5 for Artificial Intelligence and have deadlines after April are:

1. Hof University of Applied Science (Deadline: 15 April - 30 May 2025) - Course: Artificial Intelligence and Robotics M.Sc.
2. University of Regensburg (Deadline: 15 April - 1 June 2025) - Course: Human-Centred Artificial Intelligence
3. Deggendorf Institute of Technology (Deadline: Not specified, but has a course in Artificial Intelligence) - Course: MSc Artificial Intelligence and Data Science
4. Hochschule Neu ULM University (Deadline: 2 May - 15 July 2025) - Course: Artificial intelligence and data analytics 

Note that Entry 2 (University of Passau) also has a course related to Artificial Intelligence (Artificial intelligence engineering), but its deadline is 1 April- 31 May 2025, which starts before April, so it's not included in the 